In [ ]:
from imports import *
from scipy import constants
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0,3.3)
plt.rcParams['figure.autolayout'] = True

In [ ]:
datadir = "data/thermalPhonon"

# Low energy phonon spectra
temp50mK  = datadir+"/thermal-0.05K_51250814_000000.root"
temp200mK = datadir+"/thermal-0.2K_51250814_000000.root"
temp0K    = datadir+"/thermal-0K_51250814_000000.root"
temp1K    = datadir+"/thermal-1K_51250814_000000.root"
temp4K    = datadir+"/thermal-4K_51250814_000000.root"

# 100 ueV phonons 1% sidewall SMAH
temp_smah1_0K = datadir+"/thermal-anh0.01-0K_51250814_000000.root"
temp_smah1_50mK = datadir+"/thermal-anh0.01-0.05K_51250814_000000.root"
temp_smah1_200mK = datadir+"/thermal-anh0.01-0.2K_51250814_000000.root"

# 100 ueV phonons 10% sidewall SMAH
temp_smah10_0K = datadir+"/thermal-anh0.1-0K_51250814_000000.root"
temp_smah10_50mK = datadir+"/thermal-anh0.1-0.05K_51250814_000000.root"
temp_smah10_200mK = datadir+"/thermal-anh0.1-0.2K_51250814_000000.root"

# 1 keV energy deposits
edep0K    = datadir+"/iZIP7-1keV-0K_51250814_000000.root"
edep50mK  = datadir+"/iZIP7-1keV-0.05K_51250814_000000.root"
edep200mK = datadir+"/iZIP7-1keV-0.2K_51250814_000000.root"

# 1 keV ER with 1% sidewall SMAH
smah1_0K    = datadir+"/iZIP7-anh0.01-1keV-0K_51250814_000000.root"
smah1_50mK  = datadir+"/iZIP7-anh0.01-1keV-0.05K_51250814_000000.root"
smah1_200mK = datadir+"/iZIP7-anh0.01-1keV-0.2K_51250814_000000.root"

# 1 keV ER with 10% sidewall SMAH (may not work)
smah10_0K    = datadir+"/iZIP7-anh0.1-1keV-0K_51250814_000000.root"
smah10_50mK  = datadir+"/iZIP7-anh0.1-1keV-0.05K_51250814_000000.root"
smah10_200mK = datadir+"/iZIP7-anh0.1-1keV-0.2K_51250814_000000.root"

In [ ]:
def getHits(files):
    """Load mczip0 TTree from specified file or files."""
    if not files: return None
        
    branches = ["EventNum","Track","Step","KE","Yield"]
    hits = CDataFrame("G4SimDir/mczip0", files).AsNumpy(branches)
    return hits

In [ ]:
def getEfficiency(files):
    """Load g4dmcEvent TTree from specified file or files."""
    if not files: return None

    branches = ["EventNum","PhononE"]
    eff = CDataFrame("G4SimDir/g4dmcEvent", files).AsNumpy(branches)

    # Compute efficiency for 1 keV ER @ 100V = 34.784 keV
    Eexpected = (1.+4./2.96)*1e3
    eff["PhEff"] = np.sum(eff["PhononE"],axis=1)/Eexpected
    return eff

In [ ]:
def getTime(files):
    """Average CPU time per event."""
    if not files: return None

    branches = ["Events","Elapsed","UserCPU","SystemCPU"]
    times = CDataFrame("G4SettingsInfoDir/runtime", files).AsNumpy(branches)
    return times

In [ ]:
def boltzmann(energy,temperature):
    """Returns normalized Maxwell-Boltzmann for a given temperature.
       energy (can be np.array) in units of eV, temperature in kelvins."""
    if (temperature <= 0.): return energy*0.

    evToJ = constants.physical_constants["electron volt"][0]
    
    kT = constants.k * temperature/evToJ;
    coeff = 2. * np.sqrt(energy/(constants.pi*kT))
    return coeff*np.exp(-energy/kT)

In [ ]:
def drawBoltzmann(temperature, Eunits=1e6):
    """Add Maxwell-Boltzmann curve to current plot."""
    thePlot = plt.gca()
    
    # Get axis ranges and draw MB shape
    npt = 100
    emin,emax = thePlot.get_xlim()
    if (emin<0.): emin = 0.
    dE = (emax-emin)/npt
    Edata = np.arange(emin,emax,dE)
    
    Ymax = thePlot.get_ylim()[1]
    MBdata = Ymax*boltzmann(Edata/Eunits, temperature)
    thePlot.plot(Edata,MBdata)

def hitsAndKT(file,temperature,title="Phonons",maxE=None,logy='linear'):
    """Plot absorbed phonon data overlaid with Maxwell-Boltzmann."""

    ueV = 1e6      # Convert eV to microeV

    hits = getHits(file)

    # Set plot range consistent with dataset
    if not maxE: maxE = min(ueV*hits["Yield"].max(),300.)

    # Fill histogram first
    plt.hist(hits["Yield"]*ueV,bins=100,range=(0.,maxE))
    plt.gca().set_yscale(logy)
    plt.title(f"{title} with thermal killing at {temperature} K")
    plt.xlabel("Phonon energy [μeV]")

    drawBoltzmann(temperature, ueV)
    
    # Save the figure
    name = f"{datadir}/thermal{title}-{temperature}K.png"
    plt.savefig(name)

    return plt

def hitsMultiKT(files,temperatures,title="Phonons",maxE=None,logy='linear'):
    """Plot multiple sets of absorbed phonon data, with legend."""

    ueV = 1e6      # Convert eV to microeV

    for i in range(len(files)):
        hits = getHits(files[i])
        time = getTime(files[i])
        cpueff = time["UserCPU"]/time["Events"]
        label = f"{temperatures[i]} K  : {cpueff[0]:.4} s/event"
        
        if not maxE: maxE = min(ueV*hits["Yield"].max(),300.)
        plt.hist(hits["Yield"]*ueV,bins=100,range=(0.,maxE),alpha=0.4,label=label)
        
    plt.gca().set_yscale(logy)
    plt.title(f"{title} with thermal killing")
    plt.xlabel("Phonon energy [μeV]")
    plt.legend()
    
    # Save the figure
    name = f"{datadir}/thermal{title}-multiT.png"
    plt.savefig(name)

    return plt

def multiHitsAndKT(files,labels,temperature,title="Phonons",maxE=None,logy='linear'):
    """Plot multiple sets of absorbed phonon data, with legend."""

    ueV = 1e6      # Convert eV to microeV

    for i in range(len(files)):
        hits = getHits(files[i])
        time = getTime(files[i])
        cpueff = time["UserCPU"]/time["Events"]
        label = labels[i]+f"  : {cpueff[0]:.4} s/event"
        
        if not maxE: maxE = min(ueV*hits["Yield"].max(),300.)
        plt.hist(hits["Yield"]*ueV,bins=100,range=(0.,maxE),alpha=0.4,label=label)
        
    plt.gca().set_yscale(logy)
    plt.title(f"{title} with thermal killing at {temperature} K")
    plt.xlabel("Phonon energy [μeV]")
    plt.legend()

    # Add Maxwell-Boltzmann distribution after legend created
    drawBoltzmann(temperature, ueV)
    
    # Save the figure
    name = f"{datadir}/thermal{title}-{temperature}T.png"
    plt.savefig(name)

    return plt


In [ ]:
hitsAndKT(temp0K, 0.)

In [ ]:
hitsAndKT(temp50mK, 0.05)

In [ ]:
hitsAndKT(temp50mK, 0.05, "UltraLow-Phonons",10)

In [ ]:
hitsAndKT(temp200mK, 0.2)

In [ ]:
hitsAndKT(temp1K, 1)

In [ ]:
hitsAndKT(temp4K, 4)

In [ ]:
hitsAndKT(temp_smah1_0K, 0., "1ueV-SMAH0.01", logy='log')

In [ ]:
hitsAndKT(temp_smah1_50mK, 0.05, "1ueV-SMAH0.01", logy='log')

In [ ]:
hitsAndKT(temp_smah1_200mK, 0.2, "1ueV-SMAH0.01", logy='log')

In [ ]:
hitsAndKT(temp_smah10_0K, 0., "1ueV-SMAH0.1", logy='log')

In [ ]:
hitsAndKT(temp_smah10_50mK, 0.05, "1ueV-SMAH0.1", logy='log')

In [ ]:
hitsAndKT(temp_smah10_200mK, 0.2, "1ueV-SMAH0.1", logy='log')

In [ ]:
cpu0K = getTime(temp_smah10_0K)
cpu50mK = getTime(temp_smah10_50mK)
time0K = cpu0K["UserCPU"]/cpu0K["Events"]
time50mK = cpu50mK["UserCPU"]/cpu50mK["Events"]
print(f"0 K:   {time0K[0]}")
print(f"50 mK: {time50mK[0]}")

In [ ]:
smahset = (temp_smah10_0K,temp_smah10_50mK) ###,temp_smah10_200mK)
temps = (0.,0.05) ###,0.2)
hitsMultiKT(smahset, temps, "SMAH0.1", logy='log')

In [ ]:
eff0K    = getEfficiency(edep0K)
eff50mK  = getEfficiency(edep50mK)
eff200mK = getEfficiency(edep200mK)

In [ ]:
plt.hist(eff0K["PhEff"],bins=20,range=(0.85,1.05),alpha=0.4,label="0 K")
plt.hist(eff50mK["PhEff"],bins=20,range=(0.85,1.05),alpha=0.4,label="50 mK")
###plt.hist(eff200mK["PhEff"],bins=20,range=(0.85,1.05),alpha=0.4,label="200 mK")
plt.title("1 keV ER in iZIP7 with thermal killing")
plt.xlabel("Collection Efficiency")
plt.legend()
plt.savefig(datadir+"/EfficiencyVsT.png")

In [ ]:
hitsAndKT(edep0K,0,"1keV")

In [ ]:
hitsAndKT(edep50mK,0.05,"1keV")

In [ ]:
hitsAndKT(edep200mK,0.2,"1keV")

In [ ]:
hitsMultiKT((edep0K,edep50mK,edep200mK),("0 K","50 mK","200 mK"),"1keV")

In [ ]:
eff0K    = getEfficiency(smah1_0K)
eff50mK  = getEfficiency(smah1_50mK)
eff200mK = getEfficiency(smah1_200mK)

In [ ]:
plt.hist(eff0K["PhEff"],bins=20,range=(0.85,1.05),alpha=0.4,label="0 K")
plt.hist(eff50mK["PhEff"],bins=20,range=(0.85,1.05),alpha=0.4,label="50 mK")
###plt.hist(eff200mK["PhEff"],bins=20,range=(0.85,1.05),alpha=0.4,label="200 mK")
plt.title("1 keV ER in iZIP7 with 1% SMAH, thermal killing")
plt.xlabel("Collection Efficiency")
plt.legend()
plt.savefig(datadir+"/Eff-SMAH0.01_VsT.png")

In [ ]:
hitsAndKT(smah1_50mK,0.05,"1keV-SMAH0.01")

In [ ]:
hitsMultiKT((smah1_0K,smah1_50mK,smah1_200mK),("0 K","50 mK","200 mK"),"1keV-SMAH0.01")